### Notes

Perform Text Classification on the data (sentiment analysis). The tweets have been pulled from Twitter and manual tagging has been done then. The names and usernames have been given codes to avoid any privacy concerns.

Columns:
1) Location

2) Tweet At

3) Original Tweet

4) Label

- Do a word cloud 
- Use Regex 

In [30]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df = pd.read_csv('Corona_NLP_train.csv')

In [22]:
df.head(20)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative
5,3804,48756,"ÜT: 36.319708,-82.363649",16-03-2020,As news of the regions first confirmed COVID-...,Positive
6,3805,48757,"35.926541,-78.753267",16-03-2020,Cashier at grocery store was sharing his insig...,Positive
7,3806,48758,Austria,16-03-2020,Was at the supermarket today. Didn't buy toile...,Neutral
8,3807,48759,"Atlanta, GA USA",16-03-2020,Due to COVID-19 our retail store and classroom...,Positive
9,3808,48760,"BHAVNAGAR,GUJRAT",16-03-2020,"For corona prevention,we should stop to buy th...",Negative


In [24]:
df.Location.iloc[13].split()

['Chicago,', 'IL']

In [4]:
df.Sentiment.value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: Sentiment, dtype: int64

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41157 entries, 0 to 41156
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   UserName       41157 non-null  int64 
 1   ScreenName     41157 non-null  int64 
 2   Location       32567 non-null  object
 3   TweetAt        41157 non-null  object
 4   OriginalTweet  41157 non-null  object
 5   Sentiment      41157 non-null  object
dtypes: int64(2), object(4)
memory usage: 1.9+ MB


In [10]:
vc = df.Location.value_counts()
vc[vc > 100]

London                     540
United States              528
London, England            520
New York, NY               395
Washington, DC             373
United Kingdom             337
Los Angeles, CA            281
India                      268
UK                         232
Australia                  225
USA                        223
Canada                     191
England, United Kingdom    191
Toronto, Ontario           160
Global                     159
Chicago, IL                155
California, USA            153
San Francisco, CA          152
Boston, MA                 150
New York, USA              140
New Delhi, India           138
New York                   126
Atlanta, GA                126
Mumbai, India              109
Worldwide                  101
Name: Location, dtype: int64

### Notes

Need to sort the locations. Set of shorthand US states to make a US pile?

In [29]:
us_places = [ 'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
           'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
           'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
           'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
           'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY', 'United States', 'USA']

def usa_location_sort(x):
    try:
        for i in x.split(' '):
            if i in us_places:
                return 'USA'
            else:
                return x 
    except:
        return x
        
df.Location.map(usa_location_sort).value_counts()

London                           540
United States                    528
London, England                  520
USA                              419
New York, NY                     395
                                ... 
New York, NY & Menlo Park, CA      1
Covering the UK                    1
Enschede - Europa                  1
Nottingham . Hong Kong             1
header source: corsolanite         1
Name: Location, Length: 12115, dtype: int64

In [35]:
cvec = CountVectorizer(strip_accents = 'ascii', stop_words = 'english', 
                       token_pattern='\w+') 
df = cvec.fit_transform(df.OriginalTweet)

In [36]:
df

<41157x79119 sparse matrix of type '<class 'numpy.int64'>'
	with 763437 stored elements in Compressed Sparse Row format>